In [4]:
import numpy as np
import os
import shutil
import time
import glob
import os
import base64
import subprocess
from tqdm import tqdm
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, ConversationChain
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers.txt import TextParser
from langchain.memory import VectorStoreRetrieverMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import Language, RecursiveCharacterTextSplitter
from langchain.vectorstores.tiledb import TileDB
from minio import Minio
import requests

In [ ]:
type(client)

In [ ]:
client = Minio("minio-service.kubeflow.svc.cluster.local:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False,           
)

In [ ]:
bucket_name = "newtiledb"

In [ ]:
def upload_files(bucket_name, file_location, client):
    found = False  # Initialize 'found' before the try block
    try:
        found = client.bucket_exists(bucket_name)
    except Exception as e:
        print("error trying to search for MinIO Bucket:", e)
        return  # Return early since we cannot proceed without knowing if the bucket exists

    if not found:
        try:
            client.make_bucket(bucket_name)
            print("Created bucket", bucket_name)
        except Exception as e:
            print("Failed to create bucket:", e)
            return  # Return early since we cannot proceed if the bucket cannot be created
    else:
        print("Bucket", bucket_name, "exists, we won't attempt to create one")
        
    # Ensure the directory exists
    if not os.path.isdir(file_location):
        print(f"The directory {file_location} does not exist.")
        return

    # Iterate through all files in the directory
    for file_name in os.listdir(file_location):
        # Construct the full file path
        source_file = os.path.join(file_location, file_name)
        # Check if it's a file and not a directory
        if os.path.isfile(source_file):
            try:
                # Upload the file
                client.fput_object(bucket_name, file_name, source_file)
                print(f"Successfully uploaded {file_name} to bucket {bucket_name}.")
            except Exception as e:
                print(f"Failed to upload {file_name}: {e}")


In [ ]:
upload_files(bucket_name,"documentation",client)

In [ ]:
def load_docs(source_dir: str) -> list:
    """Load all documents in a the given directory."""
    fns = glob.glob(os.path.join(source_dir, "*.txt"))    
    docs = []
    for i, fn in enumerate(tqdm(fns, desc="Loading documents...")):
        docs.extend(load_doc(fn))

    return docs

In [ ]:
def process_docs(docs: list, chunk_size: int, chunk_overlap: int) -> list:
    """Load the documents and split them into chunks."""

    texts = text_splitter.split_documents(docs)
    return texts


In [ ]:
documents = process_docs(docs, chunk_size=500, chunk_overlap=0)

In [ ]:
docs = load_docs("tmp_docs")

In [ ]:
import kserve

In [ ]:
db = TileDB.from_documents(
    documents, embeddings, index_uri=index_name, index_type="FLAT")

In [ ]:
embeddings_model = "all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model)

In [ ]:
query = "tell me about vector dbs?"
docs = db.similarity_search(query)
docs[0].page_content

In [ ]:
embedding_vector = embeddings.embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
docs[0].page_content

In [ ]:
docs_and_scores = db.similarity_search_with_score(query)
docs_and_scores[0]

In [ ]:
retriever = db.as_retriever(search_type="mmr")
retriever.get_relevant_documents(query)

In [ ]:
db.max_marginal_relevance_search(query, k=2, fetch_k=10)

In [ ]:
def encode_base64(message: str):
    encoded_bytes = base64.b64encode(message.encode('ASCII'))
    return encoded_bytes.decode('ASCII'

In [14]:
import requests

data = {
  "instances": [{
    "input": "what are vectors",
    "num_docs": 4  # number of documents to retrieve
  }]
}

URL = "http://vectorstore-predictor.christensenc3526.svc.cluster.local/v1/models/vectorstore:predict"  # Adjust path as necessary

response = requests.post(URL, json=data, verify=False)  # 'verify=False' for self-signed certs
print(response)
print(response.json())
print(response.text)

<Response [200]>
{'predictions': ['Vector databases are specialized storage systems designed to efficiently store and query high-dimensional vector data, often used in machine learning, artificial intelligence, and similarity search applications. This section introduces the concept of vector databases, their importance in enabling fast and scalable vector similarity searches, and typical use cases such as image and text retrieval, and recommendation systems.\n2. Getting Started with Vector Databases', 'Vector Databases Documentation\n1. Introduction to Vector Databases', 'This guide provides a step-by-step introduction to working with vector databases, including setting up your first vector database, inserting and indexing vector data, and performing similarity searches. Through practical examples, learn the basics of vector storage, querying techniques, and how to integrate vector databases into your data pipeline.\n3. Vector Database Architecture and Indexing Techniques', 'Understand